In [1]:
import pandas as pd
import numpy as np
import random
from util import set_item, usage_masking, error_cal, error_masking, Processing, damage_sum, damage_percent, volume_checker, sampler, sampler_rate, model_checker, money_sum
from model import Kcluster, Let_model
import progressbar

In [ ]:
df = pd.read_csv('./2023공모전데이터.csv')

for num in range(500):
    Power_list = list(df.Power_Code.unique())
    model = Kcluster(Power_list, df)
    random_k = random.randint(4, 9)
    random_first_search = random.randint(30, 200)
    random_min_length = random.randint(3, 12)
    random_max_volume = (197799 / random_k) * (random.uniform(1.05, 1.3))

    result = Let_model(Power_list, model, random_k, random_first_search, random_min_length, random_max_volume, df)
    check_point = 0
    for i in result:
        total, medium, small = model_checker(i, df)
        if total >= 20000:
            check_point = check_point + 1
            
        if small >= medium:
            check_point = check_point + 1

    if check_point == 2*random_k:
        print('Collect Result')
        
        temp_df = pd.DataFrame()
        for i, value in enumerate(result):
            real_money, best_money, percent = money_sum(value, df)
            temp_df.loc[i, 'item_complex'] = value
            temp_df.loc[i, 'Total_volume'] = model_checker(value, df)
            temp_df.loc[i, 'Total_Money'] = real_money * 1000
            temp_df.loc[i, 'Total_Best_Money'] = best_money * 1000
            temp_df.loc[i, 'Money_percent'] = percent
    
        temp_df.to_csv(f'./result_{num}.csv', index=False)
    else:
        print('Not to match. ckeck_point is {}, and value is {}'. format(check_point, (2*random_k)))

In [6]:
temp = model_checker(Power_list, df)
temp

(197799.625, 112256.23999999998, 85543.38500000004)